# LLM Calls and Crafting Simple Augmented Prompts

In [1]:
from utils import (
    generate_with_single_input, 
    generate_with_multiple_input
)

## Understanding the functions to call LLMs

In [6]:
output = generate_with_single_input(
    prompt="What is the capital of France?"
)

print("Role:", output['role'])
print("Content:", output['content'])

Role: assistant
Content: The capital of France is Paris.


### `generate_with_multiple_input`

This function is designed to handle multiple input messages in a conversational context. The input format is a dictionary with two keys:

1. 'role' - the role that the message is being passed (usually assistant, system or user)
2. 'content' - the prompt

In [7]:
messages = [
    {'role': 'user', 'content': 'Hello, who won the FIFA world cup in 2018?'},
    {'role': 'assistant', 'content': 'France won the 2018 FIFA World Cup.'},
    {'role': 'user', 'content': 'Who was the captain?'}
]

output = generate_with_multiple_input(
    messages=messages,
    max_tokens=100
)

print("Role:", output['role'])
print("Content:", output['content'])

Role: assistant
Content: The captain of the French team that won the 2018 FIFA World Cup was Hugo Lloris.


### Integration with OpenAI library

In [8]:
from openai import OpenAI, DefaultHttpxClient
import httpx

In [9]:
base_url = "http://proxy.dlai.link/coursera_proxy/together/"
transport = httpx.HTTPTransport(local_address="0.0.0.0", verify=False)
http_client = DefaultHttpxClient(transport=transport)

client = OpenAI(
    api_key = '',
    base_url=base_url, 
   http_client=http_client,
)

In [10]:
messages = [
    {'role': 'user', 'content': 'Hello, who won the FIFA world cup in 2018?'},
    {'role': 'assistant', 'content': 'France won the 2018 FIFA World Cup.'},
    {'role': 'user', 'content': 'Who was the captain?'}
]

In [11]:
response = client.chat.completions.create(messages=messages, model="meta-llama/Llama-3.2-3B-Instruct-Turbo")

In [12]:
print(response)

ChatCompletion(id='o8pR91Y-57nCBj-975aec617d03cf16', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The captain of the French team that won the 2018 FIFA World Cup was Hugo Lloris.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[]), seed=5468570689635906000)], created=1756290939, model='meta-llama/Llama-3.2-3B-Instruct-Turbo', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=22, prompt_tokens=73, total_tokens=95, completion_tokens_details=None, prompt_tokens_details=None, cached_tokens=0), prompt=[])


In [13]:
print(response.choices[0].message.content)

The captain of the French team that won the 2018 FIFA World Cup was Hugo Lloris.


## Integrating Data into an LLM Prompt

In [21]:
house_data = [
    {
        "address": "123 Maple Street",
        "city": "Springfield",
        "state": "IL",
        "zip": "62701",
        "bedrooms": 3,
        "bathrooms": 2,
        "square_feet": 1500,
        "price": 230000,
        "year_built": 1998
    },
    {
        "address": "456 Elm Avenue",
        "city": "Shelbyville",
        "state": "TN",
        "zip": "37160",
        "bedrooms": 4,
        "bathrooms": 3,
        "square_feet": 2500,
        "price": 320000,
        "year_built": 2005
    }
]

### Creating the Prompt

In [22]:
# creating a layout for the houses

def house_info_layout(houses):
    layout = ''
    for house in houses:
        layout += (f"House located at {house['address']}, {house['city']}, {house['state']} {house['zip']} with "
            f"{house['bedrooms']} bedrooms, {house['bathrooms']} bathrooms, "
            f"{house['square_feet']} sq ft area, priced at ${house['price']}, "
            f"built in {house['year_built']}.\n")
    return layout

In [23]:
print(house_info_layout(house_data))

House located at 123 Maple Street, Springfield, IL 62701 with 3 bedrooms, 2 bathrooms, 1500 sq ft area, priced at $230000, built in 1998.
House located at 456 Elm Avenue, Shelbyville, TN 37160 with 4 bedrooms, 3 bathrooms, 2500 sq ft area, priced at $320000, built in 2005.



A function that generates the prompt to be passed to the Language Learning Model (LLM). The function will take a user-provided query and the available housing data as inputs to effectively address the user's query.

In [30]:
def generate_prompt(query, houses):
    houses_layout = house_info_layout(houses)
    
    PROMPT = f"""
Use the following houses information to answer users queries.
{houses_layout}
Query: {query}    
             """
    return PROMPT

In [31]:
print(generate_prompt("What is the most expensive house?", houses=house_data))


Use the following houses information to answer users queries.
House located at 123 Maple Street, Springfield, IL 62701 with 3 bedrooms, 2 bathrooms, 1500 sq ft area, priced at $230000, built in 1998.
House located at 456 Elm Avenue, Shelbyville, TN 37160 with 4 bedrooms, 3 bathrooms, 2500 sq ft area, priced at $320000, built in 2005.

Query: What is the most expensive house?    
             


In [32]:
query = "What is the most expensive house? And the bigger one?"
query_without_house_info = generate_with_single_input(prompt=query, role='user')
enhanced_query = generate_prompt(query, houses=house_data)
query_with_house_info = generate_with_single_input(prompt=enhanced_query, role='assistant')

In [33]:
# Without house info
print(query_without_house_info['content'])

The most expensive house and the largest house in the world can vary depending on the source and the criteria used to measure their value and size. However, here are some of the most notable examples:

**Most Expensive House:**

The most expensive house in the world is the Antilia building, which is the private residence of Indian business magnate Mukesh Ambani and his family. Located in Mumbai, India, Antilia is valued at over $1 billion USD.

Antilia is a 27-story building that was completed in 2010 and features:

* 168 car garage
* 50-seat movie theater
* 3 helipads
* Health spa
* Fitness center
* Swimming pool
* Staff of 600 people to maintain the property

**Largest House:**

The largest house in the world is the Biltmore Estate, which is located in Asheville, North Carolina, USA. It was built in the late 1800s for George Vanderbilt II and covers an area of over 175,000 square feet.

The Biltmore Estate features:

* 250 rooms, including 35 bedrooms, 43 bathrooms, and 65 fireplaces

In [35]:
# With house info
print(query_with_house_info['content'])

The most expensive house is the one located at 456 Elm Avenue, Shelbyville, TN 37160, priced at $320,000.

The bigger house is the one located at 456 Elm Avenue, Shelbyville, TN 37160, with 4 bedrooms, 3 bathrooms, and 2500 sq ft area.
